# Bibliotecas usadas

In [192]:
import pandas as pd
import glob
import re
from datetime import datetime, timedelta
import numpy as np

# Mashup bases "fisiologicas"

In [220]:
padrao_busca_csv = "/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments/**/*.csv"
padrao_busca_excel = "/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments/**/*.xlsx"
todos_arquivos = glob.glob(padrao_busca_csv, recursive=True) + glob.glob(padrao_busca_excel, recursive=True)
# não ta pegando os xls

ids = ["IS", "LG", "LM", "MF", "TM", "ZM", "AW", "JF", "JP", "MN", "NC", "RM"]
nao = ["01. Pilot LabEEE 20231114", "02. Pilot LabEEE 20231122"]

arquivos_fisiologicos = [arquivo for arquivo in todos_arquivos if any(sigla in arquivo for sigla in ids) and not any(nome in arquivo for nome in nao)]

In [194]:
relacao_tabela_informacao = {"A (°C)" :	["4B00000069A8E941","FE00000069AF1041","B600000069B23241","7D00000069AB4641","0F00000069AAAF41","C400000069ADCC41"],
                            "D (°C)":	["CB00000069AD3041","9400000069AA4941","2700000069AD8541","C100000069B26141","CA00000069B19041","4A00000069AC0841"],
                            "F (°C)":	["C400000069AA8341","C400000069B3DB41","8100000069AE1641","2700000069AACA41","6700000069AEFC41","1C00000069B3F941"], 
                            "H (°C)":	["B200000069AE3141","EC00000069A9CD41","3300000069AC2241","3600000069AAD741","E800000069AAC041","AD00000069AF4C41"],
                            "J (°C)":	["FA00000069AD0441","3400000069B1AE41","C700000069B34D41","4B00000069B0A841","0600000069B22741","9100000069AAEA41","6700000069AB8141"],
                            "K (°C)":	["FB00000069B28F41","7B00000069AA6A41","E500000069AA2F41","1A00000069A9BF41","FE00000069B10741","E100000069AD4641"],
                            "M (°C)":	["1B00000069B06241","8C00000069ADD241","F000000069AF6941","AA00000069AA8141","1600000069ABE741","A000000069AEBA41"],
                            "O (°C)":	["7600000069AA8541","F200000069B16D41","3000000069A9C941","BF00000069AFC741","A300000069B12241","D900000069ACA241"],
                            "Q (°C)":	["8600000069B0B141","6000000069AB3141","DB00000069A9CC41","EF00000069B23141","6900000069AE8541","4300000069B3CF41"],
                            "T (°C)":	["7600000069AB9C41","4500000069B1C841","6600000069B25C41","A200000069B0BE41","D700000069B3D541","9800000069B26241","9E00000069AE7241"],
                            "T_RH" : ["6E0000006CB57E41",	"590000006CB46641",	"4B0000006CBA7341",	"D30000006CBD4C41",	"370000006341C241",	"9C000000633F9F41"]}

nomes = ["Zac", "Joao", "Igor", "Luis", "Milena", "Ana", "Marcela", "Nathalia", "Thalita", "Liege", "Rayner", "João"]
sports = ["WALKING","CYCLING","CROSS-COUNTRY_SKIING","SKATING","RIDING","ROWING"]

In [195]:
def verifica_qual_tipo_de_csv(titulo):
    for chave, valores in relacao_tabela_informacao.items():
        if titulo in valores:
            return chave

def convertepfloat(row):
    n1 = str(row.iloc[2]).replace('.', '')
    n2 = str(row.iloc[3]).replace('.', '')
    return float(n1 + '.' + n2)

def datamento(base_de_dados_crua, base_de_dados_nova):
    dia = base_de_dados_crua.iloc[0,2]
    dia = dia.replace('-','/')
    hora_inicio = base_de_dados_crua.iloc[0,3]
    data_inicio = dia + " " + hora_inicio
    data_inicio = datetime.strptime(data_inicio, '%d/%m/%Y %H:%M:%S')
    data_fim = dia + ' ' + '20:00:00'
    data_fim = datetime.strptime(data_fim, '%d/%m/%Y %H:%M:%S')
    intervalos_tempo = pd.date_range(start=data_inicio, end=data_fim, freq='s')
    base_de_dados_nova['Date'] = intervalos_tempo[:len(base_de_dados_nova)]

def limpeza_simples(id, arquivo, coluna):
    colunas = ["Date", "uni", "u", "d"]
    dados = pd.read_csv(arquivo, skiprows=20,names=colunas).fillna('0')
    dados['id'] = id
    dados[f'{coluna}'] = dados.apply(lambda row: convertepfloat(row), axis=1)
    dados = dados.iloc[:, [0,-2,-1]]
    dados['Date'] = pd.to_datetime(dados['Date'], format='%d/%m/%y %H:%M:%S')
    return dados

def calcular_TPond(row):
    if all(row[['A (°C)', 'D (°C)', 'F (°C)', 'H (°C)', 'J (°C)', 'K (°C)', 'M (°C)', 'O (°C)', 'Q (°C)', 'T (°C)']].notnull()):
        return float(0.07 * row['T (°C)'] + 0.13 * row['Q (°C)'] + 0.19 * row['O (°C)'] + 0.12 * row['M (°C)'] + 0.12 * row['K (°C)'] + 0.12 * row['J (°C)'] + 0.05 * row['H (°C)'] + 0.06 * row['F (°C)'] + 0.08 * row['D (°C)'] + 0.06 * row['A (°C)'])
    else:
        return None

In [196]:
col_names = ['id', 'Date', 'TPond (°C)', 'A (°C)', 'D (°C)', 'F (°C)', 'H (°C)', 'J (°C)', 'K (°C)', 'M (°C)', 'O (°C)', 'Q (°C)',	'T (°C)', 'wHR (bpm)', 'cHR (bpm)', 'wU (bpm)', 'wT (°C)']
BaseDeDados = pd.DataFrame(columns=col_names)

In [197]:
for arquivo in arquivos_fisiologicos:
    ultimo_backslash = arquivo.rfind('\\')
    ultimounderscore = arquivo.rfind('_')
    titulo = arquivo[ultimo_backslash+1:ultimounderscore]
    penultimo_backslash = arquivo.rfind('\\', 0, ultimo_backslash)
    id = arquivo[penultimo_backslash+1:penultimo_backslash+3]
    if "Test+Room" in titulo or any(name in titulo for name in nomes):
        heart = pd.read_csv(arquivo).fillna('0')
        if(heart.iloc[0,1] == "OTHER_OUTDOOR"):
            pulso = pd.read_csv(arquivo, usecols=["HR (bpm)"], skiprows=2)
            pulso['id'] = id
            datamento(heart,pulso)
            pulso = pulso.rename(columns={'HR (bpm)': 'wHR (bpm)'})
            BaseDeDados = pd.concat([BaseDeDados, pulso], ignore_index=True)
        elif(heart.iloc[0,1] in sports):
            peito = pd.read_csv(arquivo, usecols=["HR (bpm)"], skiprows=2)
            datamento(heart,peito)
            peito['id'] = id
            peito = peito.rename(columns={'HR (bpm)': 'cHR (bpm)'})
            BaseDeDados = pd.concat([BaseDeDados, peito], ignore_index=True)
        elif(heart.iloc[0,1] == "False" or "True"):
            continue
        else:
            print(heart.iloc[0,1])    
    elif("_U" in arquivo) or ("_T" in arquivo) or (any(titulo in valores for valores in relacao_tabela_informacao.values())):
        colunas = ["Date", "uni", "u", "d"]
        if("_T" in arquivo):
            coluna = 'wT (°C)'
            temp = limpeza_simples(id, arquivo, coluna)
            BaseDeDados = pd.concat([BaseDeDados, temp],ignore_index=True)
        elif("_U" in arquivo):
            coluna = 'wU (bpm)'
            umid = limpeza_simples(id, arquivo, coluna)
            BaseDeDados = pd.concat([BaseDeDados, umid],ignore_index=True)
        else:
            coluna = verifica_qual_tipo_de_csv(titulo)
            temp = limpeza_simples(id, arquivo, coluna)
            BaseDeDados = pd.concat([BaseDeDados, temp],ignore_index=True)
    else:
        print(titulo)

In [200]:
BaseDeDados = BaseDeDados.groupby(['id', 'Date']).agg('mean').reset_index()

In [202]:
BaseDeDados['TPond (°C)'] = BaseDeDados.apply(calcular_TPond, axis=1)

In [205]:
BaseDeDados.to_csv('BaseDeDados.csv', index=False, encoding='utf-8')

# Mashup ambientais + "fisiologicas"

In [236]:
siglas_ambientais = ['CSLAB','CSSR','HALL','OUT','TtSR']
arquivos_ambientais = [arquivo for arquivo in todos_arquivos if any(sigla in arquivo for sigla in siglas_ambientais)]

In [250]:
def rename_cols(df, tipo):
    df = df.rename(columns={'Data;': f'{tipo} Date'})

In [251]:
for arquivo in arquivos_ambientais:
    ultimo_backslash = arquivo.rfind('\\')
    ultima_coma = arquivo.rfind('_')
    titulo = arquivo[ultimo_backslash+1:ultima_coma]
    if 'CSLAB' in titulo:
        CSLAB = pd.read_csv(arquivo)
        rename_cols(CSLAB, 'CSLAB')
        print(CSLAB)
    elif 'CSSR' in titulo:
        continue
    elif 'HALL' in titulo:
        continue
    elif 'OUT' in titulo:
        continue
    elif 'TtSR' in titulo:
        continue
    else:
        print(f'Arquivo não identificado: {titulo}')

    Data; T_ar; T_Globo; UR; Veloc_1; Veloc_2; Veloc_3
0     16/01/2023 08:25;26.26;26.55;60.33;0.8;0.59;0.78
1    16/01/2023 08:26;26.35;26.75;61.42;0.31;0.17;0.24
2    16/01/2023 08:27;26.48;26.89;62.27;0.24;0.11;0.19
3     16/01/2023 08:28;26.49;27.03;61.5;0.17;0.08;0.19
4     16/01/2023 08:29;26.36;27.08;62.35;0.18;0.07;0.2
..                                                 ...
426  16/01/2023 15:31;25.68;26.37;58.36;0.21;0.08;0.14
427    16/01/2023 15:32;25.6;26.33;58.3;0.25;0.14;0.21
428  16/01/2023 15:33;25.42;26.31;58.41;0.25;0.08;0.13
429   16/01/2023 15:34;25.61;26.31;58.04;0.2;0.08;0.14
430   16/01/2023 15:35;25.6;26.24;57.64;0.24;0.13;0.23

[431 rows x 1 columns]
    Data; T_ar; T_Globo; UR; Veloc_1; Veloc_2; Veloc_3
0    17/01/2024 09:35;  25.60; 26.34; 65.52; 0.98; ...
1    17/01/2024 09:36;  25.64; 26.37; 65.43; 0.27; ...
2    17/01/2024 09:37;  25.57; 26.38; 65.33; 0.21; ...
3    17/01/2024 09:38;  25.54; 26.29; 65.00; 0.18; ...
4    17/01/2024 09:39;  25.39; 26.19; 64.

# Mashup forms + "ambientais/fisiologicos"

In [227]:
arquivos_forms = [arquivo for arquivo in todos_arquivos if('Probral-Survey' in arquivo)]
arquivos_forms

['/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments\\Probral-Survey-v02 (Portuguese) (respostas) - Respostas ao formulário 1.csv',
 '/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments\\03. LabEEE A1 20240116\\00 DATA\\Probral-Survey-v02 (Portuguese) (respostas) - Respostas ao formulário 1.csv',
 '/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments\\Probral-Survey-v02 (Portuguese) (respostas).xlsx',
 '/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments\\02. Pilot LabEEE 20231122\\00 DATA\\20231122-Probral-Survey-v01 (Portuguese) (respostas).xlsx',
 '/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments\\03. LabEEE A1 20240116\\00 DATA\\Probral-Survey-v02 (Portuguese) (respostas).xlsx',
 '/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments\\04. LabEEE B1 20240117\\00 DATA\\Probral-Survey-v02 (Portuguese) (respostas).xlsx',
 '/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments\\05. LabEEE B2 20240122\\00 DATA\\P